In [14]:
# NOTE: コメントワードクラウド

import json
import re

import pandas as pd
import requests

from googleapiclient.discovery import build

import sys
sys.path.append("/Users/media/Code/Dev/Python/youtubeAPI/")
import video_id

import collections
import copy
import json
import re

import MeCab
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# markdown出力用
import pytablewriter
import requests
from wordcloud import WordCloud

# 必要モジュールのインポート
import os
from dotenv import load_dotenv

# .envファイルの内容を読み込見込む
load_dotenv()


API_KEY = os.environ['API_KEY']
BASE_URL = os.environ['BASE_URL']
FONT_PATH = os.environ['FONT_PATH']


# print(video_id.VIDEO_ID)

In [15]:
def get_videoTittle_from_videoId(videoId):
    youtube = build('youtube', 'v3', developerKey=API_KEY)
    videos_response = youtube.videos().list(
        part='snippet,statistics',
        id='{},'.format(videoId)
    ).execute()
    # snippet
    snippetInfo = videos_response["items"][0]["snippet"]
    # 動画タイトル
    return snippetInfo['title']

In [16]:
def get_comment_info(api_key, video_id, page_token):
    url = BASE_URL + 'commentThreads'
    param = {
        'key': api_key,
        'videoId': video_id,
        'part': 'replies, snippet',
        'maxResults': '100',
    }

    if page_token:
        param['pageToken'] = page_token

    response = requests.get(url, params=param)
    return response.json()

In [17]:
def get_video_comments(api_key, video_id):
    comments = []
    page_token = ''

    while page_token != None:
        resource = get_comment_info(api_key, video_id, page_token)

        for comment_thread in resource['items']:
            # コメント取得
            comment = comment_thread['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

            if ('replies' in comment_thread) and ('comments' in comment_thread['replies']):
                for replies in comment_thread['replies']['comments']:
                    # コメント取得
                    reply_comment = replies['snippet']['textDisplay']
                    comments.append(reply_comment)

        if 'nextPageToken' in resource:
            page_token = resource['nextPageToken']
        else:
            page_token = None
    return comments

In [22]:
# 特定の品詞の単語リストを取り出す
def get_word_class(text, kind='', min_len=1):
    word_list = []
    tagger = MeCab.Tagger()
    node = tagger.parseToNode(text)

    while node:
        word = node.surface
        pos = node.feature.split(',')[1]
        if (pos == kind) & (len(word) > min_len):
            word_list.append(word)
        node = node.next

    return word_list

In [19]:
def get_noun_list(comments):
    all_noun = []

    # NOTE: 名詞を抽出する前にテキスト整形する
    # 改行タグの削除
    comments = list(map(lambda x: x.replace('<br />', ''), comments))

    for comment in comments:
        # aタグを消す
        reg = re.compile('<a.*/a>')
        atags = reg.findall(comment)
        for atag in atags:
            comment = comment.replace(atag, '')

        all_noun.extend(get_word_class(comment, kind='一般', min_len=1))

    return all_noun

In [20]:
# %%  ある動画に対するコメントを取得、解析する
def get_wc_from_comment(video_id, video_title):
    comments = get_video_comments(API_KEY, video_id)
    all_noun = get_noun_list(comments)
    stop_word_list = ["br"]
    wc = WordCloud(background_color='white',
        font_path=FONT_PATH, width=900, height=500, collocations=False, stopwords=stop_word_list).generate(' '.join(all_noun))

    # wc.to_file('./wordcloud_{}.png'.format(video_title))
    plt.figure(figsize=(18,10))
    plt.axis("off")
    plt.imshow(wc)

In [21]:
for id in video_id.VIDEO_ID:
    get_wc_from_comment(id, get_videoTittle_from_videoId(id))

['最高', 'みこ', 'chan', 'very', 'amazing', 'don', 'know', 'what', 'it', 'is', 'about', 'this', 'song', 'that', 'makes', 'me', 'want', 'to', 'cry', 'every', 'time', 'the', 'music', 'is', 'gorgeous', 'SHE', 'is', 'gorgeous', 'and', 'HER', 'VOICE', 'is', 'so', 'gorgeous', 'and', 'don', 'even', 'know', 'what', 'the', 'song', 'is', 'about', 'haven', 'translated', 'lyrics', 'or', 'anything', 'it', 'just', 'she', 'is', 'SO', 'PRETTY', 'that', 'even', 'hearing', 'chiptune', 'version', 'of', 'this', 'in', 'Tian', 'Nya', 'video', 'today', 'put', 'me', 'in', 'tears', 'Last', 'Frontier', 'Stellar', 'かれこれ', 'br', '最初', 'br', '歌姫', 'video', 'br', 'br', 'Me', 'blue', 'zenith', '初めて', '鳥肌', '自分', 'みこ', '久し振り', '才能', '心底', '切っ掛け', 'br', 'ホロメン', 'きゅっと', '感じ', 'アド', 'this', 'is', 'the', 'power', 'of', 'an', 'ライブ', 'br', 'やっぱ', '最高', 'あと', 'vtuber', 'br', 'br', 'br', 'これぞ', '伝説', '名曲', '久しぶり', 'br', '最後', '左側', 'なか', '場面', '思いっきり', '地元', 'br', 'ランニング', 'br', 'ペース', '最高', 'stellar', '響き', 'br', 'ガン', 'ダム', '魔

TypeError: can only join an iterable